# Imports

In [1]:
from pathlib import Path
import random

import numpy as np
import torch
from PIL import Image
from transformers import AutoProcessor, MllamaForConditionalGeneration

# Set Random Seed for Reproducibility

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✅ Random seed set to 42 for reproducibility")

✅ Random seed set to 42 for reproducibility


# Load the model

In [3]:
# Update this path to your local Llama model
# model_id = "/home/jovyan/shared_PTM/Llama-3.2-11B-Vision-Instruct"
model_id = "/home/jovyan/nfs_share/models/Llama-3.2-11B-Vision-Instruct"

print("🔧 Loading Llama-3.2-Vision model...")
model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

# processor

🔧 Loading Llama-3.2-Vision model...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

# Load the image

In [4]:
# Update this path to your test image
# imageName = "/home/jovyan/shared_PoC_data/evaluation_data/image_009.png"
imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/image_008.png"
print("📁 Loading image...")
image = Image.open(imageName)

# CRITICAL: Store as list for multi-turn compatibility
images = [image]

print(f"✅ Image loaded: {image.size}")
print(f"✅ Images list created with {len(images)} image(s)")

📁 Loading image...
✅ Image loaded: (900, 1320)
✅ Images list created with 1 image(s)


# Define the prompt

In [5]:
# Visual Question Answering - ask a simple question about the image
prompt = """
You are an expert document analyser specializing in Date Grouped Australian Bank Statement extraction.
Date Grouped Bank Statements are date ordered, with one or more transactions for each date header.
Every transaction for a given date heading has a description, a debit/credit amount and finally a balance amount with a ' CR' suffix.
Extract all balance amounts along with their ' CR' suffix, the transaction dates (from the date heading) and transaction descriptions,
maintaining the same date ordering as the image, with every transaction appearing on its own row and remembering that some date headings have more than one balance.
"""

# Create message structure for Llama
messageDataStructure = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {
                "type": "text",
                "text": prompt,
            },
        ],
    }
]

print(f"💬 Prompt: {prompt}")
print("🤖 Generating response with Llama-3.2-Vision...")

💬 Prompt: 
You are an expert document analyser specializing in Date Grouped Australian Bank Statement extraction.
Date Grouped Bank Statements are date ordered, with one or more transactions for each date header.
Every transaction for a given date heading has a description, a debit/credit amount and finally a balance amount with a ' CR' suffix.
Extract all balance amounts along with their ' CR' suffix, the transaction dates (from the date heading) and transaction descriptions,
maintaining the same date ordering as the image, with every transaction appearing on its own row and remembering that some date headings have more than one balance.

🤖 Generating response with Llama-3.2-Vision...


# Clean Response Function

In [6]:
def clean_llama_response(response: str) -> str:
    """Remove chat template artifacts and extract only the assistant's response.
    
    Note: This function is kept for backwards compatibility, but when using
    the proper multi-turn pattern (trimming generate_ids), it's not needed.
    """
    start_marker = "<|start_header_id|>assistant<|end_header_id|>"
    end_marker = "<|eot_id|>"
    
    start_idx = response.find(start_marker)
    if start_idx != -1:
        start_idx += len(start_marker)
        end_idx = response.find(end_marker, start_idx)
        if end_idx != -1:
            return response[start_idx:end_idx].strip()
    
    return response.strip()

# Process the prompt

In [7]:
# Process the input using the CORRECT multi-turn pattern
# Based on: https://medium.com/data-science/chat-with-your-images-using-multimodal-llms-60af003e8bfa

textInput = processor.apply_chat_template(
    messageDataStructure, add_generation_prompt=True
)

# CRITICAL: Use named parameter 'images=' with list
inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# Generate response with deterministic parameters
output = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=False,
    temperature=None,
    top_p=None,
)

# CRITICAL: Trim input tokens from output (this is the key to clean responses!)
generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
cleanedOutput = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

print("✅ Response generated successfully!")
print("\n" + "=" * 60)
print("CLEANED EXTRACTION:")
print("=" * 60)
print(cleanedOutput)
print("=" * 60)

# Save the cleaned response to a file
output_path = Path("llama_grouped_bank_statement_output.txt")

with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write(cleanedOutput)

print(f"✅ Response saved to: {output_path}")
print(f"📊 File size: {output_path.stat().st_size} bytes")

✅ Response generated successfully!

CLEANED EXTRACTION:
Here is the extracted data in the format you requested:

| Date | Balance | Transaction Description |
| --- | --- | --- |
| 07/09/2025 | $48890.58 | EFTPOS Cash Out PRICELINE PHARMACY MACKAY QLD |
| 08/09/2025 | $49212.76 | EFTPOS Purchase OFFICEWORKS BUSINESS ROCKHAMPTO... |
| 04/09/2025 | $49277.09 | Mortgage Repayment MORT 0103P16754533 NAB |
| 03/09/2025 | $49226.81 | OSKO Payment to MIKE CHEN 809845773133 |
| 03/09/2025 | $51312.57 | Direct Debit 15395P41608263 MWF 42730 |
| 02/09/2025 | $61511.48 | Salary Payment ATO 28782P60782100 |
| 01/09/2025 | $43405.77 | Auto Payment UTILITIES AGL 79479A0P85943944 |
| 01/09/2025 | $43440.81 | DIRECT CREDIT SALARY 6080P67812986 |
| 01/09/2025 | $43708.81 | BPAY Payment BILLER60279 CRN 9813018480 |
| 31/08/2025 | $39296.85 | Professional Services Red Energy 98073P31959417 |
| 29/08/2025 | $41754.72 | Card Purchase RED ROOSTER Paramatta SA |
| 29/08/2025 | $42025.03 | Credit Card Payment 

## Multi-Turn Conversation Support

Llama supports multi-turn conversations by maintaining a conversation history list:

## 🔑 Key Multi-Turn Pattern for Llama 3.2 Vision

This notebook uses the **correct multi-turn conversation pattern** discovered from the Medium article:
[Chat with Your Images Using Llama 3.2-Vision Multimodal LLMs](https://medium.com/data-science/chat-with-your-images-using-multimodal-llms-60af003e8bfa)

### Critical Requirements:

1. **Images as List**: `images = [image]` (not just `image`)
2. **Named Parameter**: `processor(images=images, text=text, ...)` (not positional args)
3. **Trim Generated Tokens**: `generate_ids[:, inputs['input_ids'].shape[1]:-1]`
4. **Same Images Every Turn**: Pass the same `images` list for all turns

### Message Structure:

- **Turn 1**: `{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": "..."}]}`
- **Turn 2+**: `{"role": "user", "content": [{"type": "text", "text": "..."}]}` (no image in content)
- **Assistant**: `{"role": "assistant", "content": [{"type": "text", "text": "..."}]}`

The model attends to the image only in the first turn, but the processor needs the images list for all turns because the chat template contains the `<|image|>` token.

In [8]:
# Store conversation history for multi-turn support
# Initialize with first exchange
conversation_history = messageDataStructure.copy()

# Add assistant's response to history
conversation_history.append({
    "role": "assistant",
    "content": [{"type": "text", "text": cleanedOutput}]
})

print("✅ Conversation history initialized")
print(f"📊 Current conversation has {len(conversation_history)} messages (1 user + 1 assistant)")
print(f"💡 Pattern: Using working multi-turn approach from Medium article")

✅ Conversation history initialized
📊 Current conversation has 2 messages (1 user + 1 assistant)
💡 Pattern: Using working multi-turn approach from Medium article


### Debug: View Conversation Context

This cell helps you see what's being sent to the model:

In [9]:
# Optional: Debug conversation structure
print("🔍 Current conversation structure:")
print("=" * 60)
for i, msg in enumerate(conversation_history, 1):
    print(f"\nMessage {i} ({msg['role']}):")
    for content in msg['content']:
        if content['type'] == 'text':
            preview = content['text'][:100] + "..." if len(content['text']) > 100 else content['text']
            print(f"  [text]: {preview}")
        else:
            print(f"  [{content['type']}]")
print("=" * 60)

🔍 Current conversation structure:

Message 1 (user):
  [image]
  [text]: 
You are an expert document analyser specializing in Date Grouped Australian Bank Statement extracti...

Message 2 (assistant):
  [text]: Here is the extracted data in the format you requested:

| Date | Balance | Transaction Description ...


### Follow-up Question (Turn 2)

In [10]:
# Follow-up question (Turn 2)
# Using the WORKING pattern from: https://medium.com/data-science/chat-with-your-images-using-multimodal-llms-60af003e8bfa

follow_up_prompt = "How many transactions are shown in this bank statement?"

# Append user's follow-up to conversation history (text only - NO image in content)
conversation_history.append({
    "role": "user",
    "content": [{"type": "text", "text": follow_up_prompt}]
})

print(f"💬 Follow-up question: {follow_up_prompt}")
print("🤖 Generating follow-up response with Llama-3.2-Vision...")

# Process with updated conversation history
textInput = processor.apply_chat_template(
    conversation_history, add_generation_prompt=True
)

# CRITICAL: Use named parameter 'images=' and pass the SAME images list
inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# Generate response
output = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=False,
    temperature=None,
    top_p=None,
)

# CRITICAL: Trim input tokens from output
generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
cleanedOutput2 = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

print("\n✅ Follow-up response generated successfully!")
print("\n" + "=" * 60)
print("FOLLOW-UP RESPONSE:")
print("=" * 60)
print(cleanedOutput2)
print("=" * 60)

# Update conversation history with assistant's response
conversation_history.append({
    "role": "assistant",
    "content": [{"type": "text", "text": cleanedOutput2}]
})

print(f"\n📊 Conversation now has {len(conversation_history)} messages")

💬 Follow-up question: How many transactions are shown in this bank statement?
🤖 Generating follow-up response with Llama-3.2-Vision...

✅ Follow-up response generated successfully!

FOLLOW-UP RESPONSE:
There are 40 transactions shown in this bank statement.

📊 Conversation now has 4 messages


### Additional Follow-up (Turn 3 - Optional)

You can continue the conversation by running this cell with different questions:

In [11]:
# Third turn - another follow-up (uncomment to use)
follow_up_prompt_3 = "What is the date range covered by this bank statement?"

# Append user's follow-up to conversation history
conversation_history.append({
    "role": "user",
    "content": [{"type": "text", "text": follow_up_prompt_3}]
})

print(f"💬 Follow-up question: {follow_up_prompt_3}")
print("🤖 Generating follow-up response with Llama-3.2-Vision...")

# Process with updated conversation history
textInput = processor.apply_chat_template(
    conversation_history, add_generation_prompt=True
)

# Use named parameter 'images=' and pass the SAME images list
inputs = processor(images=images, text=textInput, return_tensors="pt").to(model.device)

# Generate response
output = model.generate(
    **inputs,
    max_new_tokens=2000,
    do_sample=False,
    temperature=None,
    top_p=None,
)

# Trim input tokens from output
generate_ids = output[:, inputs['input_ids'].shape[1]:-1]
cleanedOutput3 = processor.decode(generate_ids[0], clean_up_tokenization_spaces=False)

print("\n✅ Follow-up response generated successfully!")
print("\n" + "=" * 60)
print("FOLLOW-UP RESPONSE:")
print("=" * 60)
print(cleanedOutput3)
print("=" * 60)

# Update conversation history with assistant's response
conversation_history.append({
    "role": "assistant",
    "content": [{"type": "text", "text": cleanedOutput3}]
})

print(f"\n📊 Conversation now has {len(conversation_history)} messages")
print("\n💡 To ask more questions, copy this cell and modify the 'follow_up_prompt_3' variable")

💬 Follow-up question: What is the date range covered by this bank statement?
🤖 Generating follow-up response with Llama-3.2-Vision...

✅ Follow-up response generated successfully!

FOLLOW-UP RESPONSE:
The date range covered by this bank statement is from **August 8, 2025**, to **September 7, 2025**.

📊 Conversation now has 6 messages

💡 To ask more questions, copy this cell and modify the 'follow_up_prompt_3' variable


### Save Multi-Turn Conversation

In [12]:
# Save the entire conversation to a file
output_path = Path("llama_multiturn_conversation.txt")

with output_path.open("w", encoding="utf-8") as text_file:
    text_file.write("=" * 60 + "\n")
    text_file.write("MULTI-TURN CONVERSATION WITH LLAMA-3.2-VISION\n")
    text_file.write("=" * 60 + "\n\n")
    
    for i, msg in enumerate(conversation_history, 1):
        role = msg["role"].upper()
        text_file.write(f"\n{'-' * 60}\n")
        text_file.write(f"MESSAGE {i} - {role}\n")
        text_file.write(f"{'-' * 60}\n\n")
        
        for content in msg["content"]:
            if content["type"] == "text":
                text_file.write(content["text"] + "\n")
            elif content["type"] == "image":
                text_file.write("[IMAGE]\n")
    
    text_file.write("\n" + "=" * 60 + "\n")
    text_file.write(f"Total messages: {len(conversation_history)}\n")
    text_file.write("=" * 60 + "\n")

print(f"✅ Full conversation saved to: {output_path}")
print(f"📊 File size: {output_path.stat().st_size} bytes")
print(f"💬 Total messages in conversation: {len(conversation_history)}")

✅ Full conversation saved to: llama_multiturn_conversation.txt
📊 File size: 5079 bytes
💬 Total messages in conversation: 6
